In [1]:
import requests
import json
import pandas as pd 
import time
from bs4 import BeautifulSoup as bs

In [2]:
connection_timeout = 30 # seconds

In [3]:
def make_soup(url):
    content = requests.get(url)
    html_content=content.content
    soup=bs(html_content,'html.parser')
    
    return soup

In [4]:
url = "https://www.newagebd.net/tags/Road%20accident/"
base_url = "https://www.newagebd.net/"

In [5]:
soup = make_soup(url)

In [6]:
links = []
headings = []

In [7]:
start_time = time.time()

for i in range(68):  #total 68
    try:
        print("Page:", i)
        container = soup.find("div", attrs={"class": "categoryContent"})
        for row in container.find_all('h3'):
            # getting the heading
            heading = row.text
            headings.append(heading)

            # getting the link to the article
            link = row.find('a')
            if 'href' in link.attrs:
                #print(f"{heading} - {link.attrs['href']}")
                links.append(link.attrs['href'])

        try:
            next_button = soup.find(text="Next ›")
            next_button_link = soup.find(text="Next ›").parent.attrs['href']
            soup = make_soup(next_button_link)
        except AttributeError:
            print("End of Pages.")
    
    except ConnectionError:
        if time.time() > start_time + connection_timeout:
            raise Exception('Unable to get updates after {} seconds of ConnectionErrors'.format(connection_timeout))
        else:
            time.sleep(1) # attempting once every second

Page: 0
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41
Page: 42
Page: 43
Page: 44
Page: 45
Page: 46
Page: 47
Page: 48
Page: 49
Page: 50
Page: 51
Page: 52
Page: 53
Page: 54
Page: 55
Page: 56
Page: 57
Page: 58
Page: 59
Page: 60
Page: 61
Page: 62
Page: 63
Page: 64
Page: 65
Page: 66
Page: 67
End of Pages.


In [8]:
links

['https://www.newagebd.net/article/142693/five-killed-five-injured-in-tangail-road-accident',
 'https://www.newagebd.net/article/142383/three-killed-in-chattogram-road-accidents',
 'https://www.newagebd.net/article/142373/three-youths-killed-in-satkhira-road-accident',
 'https://www.newagebd.net/article/142269/two-killed-in-chattogram-road-accidents',
 'https://www.newagebd.net/article/142176/2-killed-5-wounded-in-cumilla-road-accident',
 'https://www.newagebd.net/article/142167/four-killed-in-jashore-road-crash',
 'https://www.newagebd.net/article/141541/one-killed-five-injured-in-sunamganj-road-accident',
 'https://www.newagebd.net/article/141433/two-people-killed-in-ctg-road-accident',
 'https://www.newagebd.net/article/141230/three-including-minor-girl-killed-in-bogura-road-accident',
 'https://www.newagebd.net/article/141227/cop-killed-in-dhaka-road-accident',
 'https://www.newagebd.net/article/141223/woman-killed-her-daughter-injured-in-dhaka-road-accident',
 'https://www.newageb

In [9]:
len(links)

677

In [10]:
headings[676]

' Two picnickers killed in road accident in Bagerhat '

In [11]:
locations = []
published_dates = []
articles = []

In [16]:
for i in range(len(links)):
    try:
        page_soup = make_soup(links[i])
        print(i)
        top = page_soup.find("div", attrs={"class": "postInnerTopIn"})
        info_string = top.find('p').text
        split_string = info_string.split()

        try:
            index = split_string.index('|')
            locations.append(split_string[index - 1])
        except ValueError:
            locations.append("Manually add location")

        published_index = split_string.index('Published:')
        published_dates.append(split_string[published_index+2] + " " + split_string[published_index+3])

        article =  page_soup.find('div', attrs={"class": "postPageTestIn"})
        paragraphs = article.find_all('p')
        subheading = paragraphs[0].text

        article_text = ""
        for i, paragraph in enumerate(paragraphs):
            if i == 0:
                #print(paragraph.text)
                subheading = paragraph
            else:
                article_text += paragraph.text
        articles.append(article_text)
        
    except ConnectionError:
        if time.time() > start_time + connection_timeout:
            raise Exception('Unable to get updates after {} seconds of ConnectionErrors'.format(connection_timeout))
        else:
            time.sleep(1) # attempting once every second

137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386


In [17]:
len(links)

677

In [18]:
len(locations)

677

In [19]:
len(published_dates)

677

In [20]:
len(articles)

677

In [21]:
len(headings)

677

In [22]:
headings[0]

' Five killed, five injured in Tangail road accident '

In [23]:
dataset={'Newspaper Name':'New Age BD', 'Published date': published_dates ,'Header':headings , 'Article':articles ,'Newspaper Link':links}


In [24]:
import pandas as pd

In [25]:
df=pd.DataFrame(dataset)

In [26]:
df.head()

,Newspaper Name,Published date,Header,Article,Newspaper Link
0,New Age BD,"Jul 03,2021","Five killed, five injured in Tangail road acc...",At least five people were killed and five othe...,https://www.newagebd.net/article/142693/five-k...
1,New Age BD,"Jun 30,2021",Three killed in Chattogram road accidents,The deceased were identified as autorickshaw d...,https://www.newagebd.net/article/142383/three-...
2,New Age BD,"Jun 30,2021",Three youths killed in Satkhira road accident,The incident took place at about 10:00pm.The d...,https://www.newagebd.net/article/142373/three-...
3,New Age BD,"Jun 29,2021",Two killed in Chattogram road accidents,"The deceased were identified as Badsha Mia, 50...",https://www.newagebd.net/article/142269/two-ki...
4,New Age BD,"Jun 28,2021","2 killed, 5 wounded in Cumilla road accident",The identities of the deceased and injured per...,https://www.newagebd.net/article/142176/2-kill...


In [27]:
df.to_csv('NewAgeBD.csv')